# Блок 1
Подключаю библиотеки, открываю в Chrome общую страницу с вакансиями

In [1]:
#!pip install selenium bs4

In [2]:
from selenium import webdriver

from bs4 import BeautifulSoup
import pandas as pd
import re
import time

In [3]:
br = webdriver.Chrome()

br.get("https://geekjob.ru/vacancies")

The chromedriver version (142.0.7444.162) detected in PATH at chromedriver.EXE might not be compatible with the detected chrome version (143.0.7499.41); currently, chromedriver 143.0.7499.42 is recommended for chrome 143.*, so it is advised to delete the driver in PATH and retry


Список URL с разными запросами, по которым собираются все подходящие вакансии – DS, ML, аналитик данных и прочее

In [4]:
search_urls = [
    "https://geekjob.ru/vacancies?qs=data%20analyst",
    "https://geekjob.ru/vacancies?qs=data%20scientist",
    "https://geekjob.ru/vacancies?qs=ml%20engineer",
    "https://geekjob.ru/vacancies?qs=data%20engineer",
    "https://geekjob.ru/vacancies?qs=%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA%20%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85",
    "https://geekjob.ru/vacancies?qs=%D0%B1%D0%B8%D0%B7%D0%BD%D0%B5%D1%81%20%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA",
    "https://geekjob.ru/vacancies?qs=ML"
]

# Блок 2 
Функция ниже проверяет, подходит ли название вакансии под ключевые слова

In [5]:
def is_good_title(title):
    if title is None:
        return False
    
    t = title.lower()
    
    good = [
        "аналит", 
        "analyst",
        "data ",
        " data",
        "machine learning",
        "ml ", "ml-",
        "data engineer",
        "инженер данных"
    ]
    
    bad = [
        "devops", "sre", "qa", "тест", "test",
        "developer", "разработчик",
        "frontend", "back-end", "backend", "fullstack",
        "smm", "маркет", "pr", "контент", "editor",
        "sales", "продаж", "менеджер", "manager",
        "designer", "дизайн", "hr"
    ]
    
    for b in bad:
        if b in t:
            return False
    
    for g in good:
        if g in t:
            return True
    
    return False

Собираю ссылки на вакансии с поисковых страниц из списка search_urls

In [6]:
vacancy_links = []

for url in search_urls:
    print("Страница поиска:", url)
    br.get(url)
    br.implicitly_wait(3)
    time.sleep(2)
    
    page = br.page_source
    soup = BeautifulSoup(page, "html.parser")
    
    a_tags = soup.find_all("a", href=True)
    
    for a in a_tags:
        href = a.get("href")
        if href is None:
            continue
        
        if "/vacancy/" not in href:
            continue
        
        title = a.text.strip()
        if title == "":
            continue
        
        if not is_good_title(title):
            continue
        
        if href.startswith("http"):
            full_link = href
        else:
            full_link = "https://geekjob.ru" + href
        
        if full_link not in vacancy_links:
            vacancy_links.append(full_link)
    
    time.sleep(1)

print("Всего ссылок после фильтрации по названию:", len(vacancy_links))

Страница поиска: https://geekjob.ru/vacancies?qs=data%20analyst
Страница поиска: https://geekjob.ru/vacancies?qs=data%20scientist
Страница поиска: https://geekjob.ru/vacancies?qs=ml%20engineer
Страница поиска: https://geekjob.ru/vacancies?qs=data%20engineer
Страница поиска: https://geekjob.ru/vacancies?qs=%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA%20%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85
Страница поиска: https://geekjob.ru/vacancies?qs=%D0%B1%D0%B8%D0%B7%D0%BD%D0%B5%D1%81%20%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA
Страница поиска: https://geekjob.ru/vacancies?qs=ML
Всего ссылок после фильтрации по названию: 49


In [7]:
# Функция разбора зарплат
def parse_salary(text):
    if text is None:
        return None, None

    t = str(text)
    # базовая нормализация
    t = t.replace("\xa0", " ")
    t = t.replace("–", "-")
    t = t.replace("−", "-")

    new_t = ""
    for i in range(len(t)):
        ch = t[i]
        if ch == " ":
            prev_digit = i > 0 and t[i - 1].isdigit()
            next_digit = i + 1 < len(t) and t[i + 1].isdigit()
            if prev_digit and next_digit:
                continue
        new_t = new_t + ch

    t = new_t
    t_low = t.lower()

    currency = None
    if ("руб" in t_low) or ("₽" in t):
        currency = "RUB"
    elif ("usd" in t_low) or ("$" in t):
        currency = "USD"
    elif ("eur" in t_low) or ("€" in t):
        currency = "EUR"

    numbers = []
    current = ""

    for ch in t:
        if ch.isdigit():
            current = current + ch
        else:
            if current != "":
                numbers.append(int(current))
                current = ""

    if current != "":
        numbers.append(int(current))

    if len(numbers) == 0:
        return None, currency

    if ("k" in t_low or "к" in t_low or "тыс" in t_low) and max(numbers) < 1000:
        numbers = [x * 1000 for x in numbers]

    if len(numbers) == 1:
        salary = numbers[0]
        return salary, currency

    salary = min(numbers)

    return salary, currency

In [8]:
# Функция подготовки опыта
# - более 5 лет -> "5"
# - менее 1 года -> "1"
# - 1–3 года -> "1-3"
# - 3–5 лет -> "3-5"
# - любые значения -> "0"


def map_experience_to_range(exp):
    if exp is None:
        return "0"
    if isinstance(exp, str):
        s = exp.strip().lower()
        if s in {"любой опыт", "любая"}:
            return "0"
    else:
        s = str(exp)

    s = s.replace("–", "-").replace("—", "-")
    nums = re.findall(r"\d+\.?\d*", s)
    if not nums:
        return "0"

    # один номер
    if len(nums) == 1:
        return nums[0]

    # диапазон
    a = float(nums[0])
    b = float(nums[1])

    if a >= 1 and b <= 3:
        return "1-3"
    if a >= 3 and b <= 5:
        return "3-5"
    if a > 5:
        return "5"
    if b < 1:
        return "0-1"

Списки для сбора ГЕО и списки для данных

In [9]:
country_keywords = [
    "россия", "рф", "беларусь", "республика беларусь",
    "казахстан", "армения", "кипр", "сербия",
    "uae", "oae", "germany", "france", "cyprus"
]

city_keywords = [
    "москва", "санкт-петербург", "петербург", "спб",
    "минск", "лимассол", "лимасол", "чебоксары",
    "иннополис", "ереван", "нови-сад", "саратов",
    "орёл", "иннополис"
]


job_words = [
    "аналитик", "analyst",
    "engineer", "инженер",
    "developer", "разработчик",
    "менеджер", "manager",
    "дизайнер", "designer",
    "devops", "маркетолог",
    "qa", "тестировщик",
    "product", "продукт",
    "python", "backend", "frontend", "fullstack"
]


In [10]:
names = []
salary_col = []
currency_col = []
experience_col = []
location_col = []
skills_col = []
link_col = []
industry_col = []
source_col = []

# Блок 3
Парсинг

In [11]:
for url in vacancy_links:
    print("Парсим вакансию:", url)
    br.get(url)
    br.implicitly_wait(3)
    time.sleep(1)

    page = br.page_source
    soup = BeautifulSoup(page, "html.parser")

    # Собираю весь текст построчно
    all_text = soup.get_text("\n", strip=True)
    lines = all_text.split("\n")

    # Название вакансии
    current_name = None
    h1_tag = soup.find("h1")
    if h1_tag is not None:
        current_name = h1_tag.text.strip()

    # Зарплата
    current_salary_text = None
    span_tags = soup.find_all("span")

    for span in span_tags:
        txt = span.text.strip()
        low = txt.lower()
        if (
            ("₽" in txt)
            or ("руб" in low)
            or ("$" in txt)
            or ("usd" in low)
            or ("€" in txt)
            or ("eur" in low)
        ):
            current_salary_text = txt
            break

    salary, curr = parse_salary(current_salary_text)

    # Опыт

    experience = None
    for line in lines:
        if "Опыт работы" in line or "Опыт:" in line:
            experience = map_experience_to_range(line.strip())

            break

    # Индустрия
    current_industry = None
    for i in range(len(lines)):
        if "Отрасль и сфера применения" in lines[i]:
            if i + 1 < len(lines):
                current_industry = lines[i + 1].strip()
            break

    # Навыки
    current_skills = None
    for i in range(len(lines)):
        if "Специализация" in lines[i]:
            if i + 1 < len(lines):
                current_skills = lines[i + 1].strip()
            break

    # Локация
    current_location = None
    location_candidates = []

    for line in lines:
        txt = line.strip()
        if txt == "":
            continue

        low = txt.lower()

        # Мусор
        if "войти/регистрация" in low:
            continue
        if "geekjob" in low:
            continue
        if "вакансия " in low:
            continue
        if "откликнуться" in low:
            continue
        if "опыт работы" in low:
            continue
        if "зарплат" in low:
            continue

        is_job = False
        for w in job_words:
            if w in low:
                is_job = True
                break
        if is_job:
            continue

        if len(txt) > 60:
            continue

        location_candidates.append(txt)

    for txt in location_candidates:
        if "/" in txt and len(txt) <= 40:
            low = txt.lower()

            has_loc_word = False
            for kw in country_keywords + city_keywords:
                if kw in low:
                    has_loc_word = True
                    break

            if not has_loc_word:
                continue

            current_location = txt.strip()
            break

    if current_location is None:
        for txt in location_candidates:
            low = txt.lower()
            has_loc_word = False

            for kw in country_keywords + city_keywords:
                if kw in low:
                    has_loc_word = True
                    break

            if has_loc_word:
                current_location = txt.strip()
                break
    if current_location is not None:
        current_location = current_location.replace(",", "/")

    # Записываю в списки
    names.append(current_name)
    salary_col.append(salary)
    currency_col.append(curr)
    experience_col.append(experience)
    location_col.append(current_location)
    skills_col.append(current_skills)
    link_col.append(url)
    industry_col.append(current_industry)
    source_col.append("geekjob")

    time.sleep(1)


Парсим вакансию: https://geekjob.ru/vacancy/69383f88e6cc381065092342
Парсим вакансию: https://geekjob.ru/vacancy/6911f051e74ba724ff093dd8
Парсим вакансию: https://geekjob.ru/vacancy/69247652224f5aa99d09bd40
Парсим вакансию: https://geekjob.ru/vacancy/693c06ebe288ea48db0c6764
Парсим вакансию: https://geekjob.ru/vacancy/6913a86143f9ab26710c2f5f
Парсим вакансию: https://geekjob.ru/vacancy/691dbcf012bbfd2e77072f25
Парсим вакансию: https://geekjob.ru/vacancy/693c0ab95dcc1817cc07a093
Парсим вакансию: https://geekjob.ru/vacancy/69390be0fb140711b30d8e37
Парсим вакансию: https://geekjob.ru/vacancy/69383998a77789b7630d49f2
Парсим вакансию: https://geekjob.ru/vacancy/69380bbdb4849af83f0a0ff3
Парсим вакансию: https://geekjob.ru/vacancy/692719281b2e75e81e088f76
Парсим вакансию: https://geekjob.ru/vacancy/6936c2aa747697ea740a2a92
Парсим вакансию: https://geekjob.ru/vacancy/6932a034dbb3b32ec20ce982
Парсим вакансию: https://geekjob.ru/vacancy/693127af1430309b2a058bc3
Парсим вакансию: https://geekjob.r

# Блок №4
Сбор в датафрейм

In [12]:
data = {
    "name": names,
    "salary": salary_col,
    "currency": currency_col,
    "experience": experience_col,
    "location": location_col,
    "skills": skills_col,
    "link": link_col,
    "industry": industry_col,
    "source": source_col,
}

df = pd.DataFrame(data)

# Убираю дубли
df = df.drop_duplicates(subset="link")

df.to_csv("geekjob_vacancies.csv", index=False)

In [13]:
br.quit()

In [14]:
df.head()

,name,salary,currency,experience,location,skills,link,industry,source
0,Lead Data Analyst,3000.0,USD,3-5,Санкт-Петербург/ Россия,"Аналитика, Data Science, Big Data",https://geekjob.ru/vacancy/69383f88e6cc3810650...,Логистика и транспорт,geekjob
1,System Analyst,9000.0,USD,3-5,Санкт-Петербург,"Аналитика, Data Science, Big Data",https://geekjob.ru/vacancy/6911f051e74ba724ff0...,SaaS/PaaS,geekjob
2,Web & App Analyst,3000.0,EUR,1-3,Санкт-Петербург,"Аналитика, Data Science, Big Data",https://geekjob.ru/vacancy/69247652224f5aa99d0...,"Консалтинг, профессиональные услуги",geekjob
3,Dana Analyst,140000.0,RUB,5,не РФ/РБ,"Аналитика, Data Science, Big Data",https://geekjob.ru/vacancy/693c06ebe288ea48db0...,Data Analysis,geekjob
4,Head of Analytics / Lead Analyst (Fintech),500000.0,RUB,3-5,Удалённо (исключая РФ и Беларусь),"Аналитика, Data Science, Big Data",https://geekjob.ru/vacancy/6913a86143f9ab26710...,Банковская и страховая сфера,geekjob
